In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

ВВОДНЫЕ УСЛОВИЯ

In [39]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [40]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X.T, y)
model.predict(X.T)

array([0.2617132 , 0.17509907, 1.02794228, 0.21840613, 0.53795688,
       0.31862798, 1.20396413, 0.05324898, 0.28336673, 0.91967463])

In [41]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [42]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [43]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [44]:
def calc_logloss(y, y_pred):                                                      # Исходный код функции
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [45]:
y_pred=sigmoid(model.predict(X.T))
y_pred

array([0.56505739, 0.54366327, 0.73651677, 0.55438552, 0.63133701,
       0.57898984, 0.76922923, 0.5133091 , 0.57037143, 0.7149758 ])

In [46]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    if i % (iterations / 10) == 0:
        print(i, W, err)
    return W

1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [47]:
def calc_logloss_mod(y, y_pred):                                                              # Измененная функция
    y_pred_res=np.where(y_pred==1, y_pred-1e-07, np.where(y_pred==0, y_pred+1e-07, y_pred))
    err = - np.mean(y * np.log(y_pred_res) + (1.0 - y) * np.log(1.0 - y_pred_res))
    return err

In [48]:
y_pred1=np.array([0.56505739, 0., 0.73651677, 0.55438552, 1.,
       0.57898984, 0.76922923, 0.5133091 , 0., 0.7149758 ])
calc_logloss_mod(y, y_pred1)

2.024785862818215

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [49]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss_mod(y, y_pred) 
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    if i % (iterations / 10) == 0:
        print(i, W, err)
    return W

In [50]:
W = eval_model(X_st, y, iterations=1000, alpha=1e-5)

1000 [ 0.49282757 -0.15007512  0.64748969  1.51727928] 1.2013133809011647


In [51]:
def eval_model(X, y, verbose=False, alpha=1e-4, tol=0.00001): 
    view_ind = 10**(-np.log10(tol)-2) if -np.log10(tol)-2>=1 else 1 
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    min_err = float('inf')  
    n_iter = 0 
    stop_chek = True
    errors = []  
    while stop_chek:
        n_iter += 1
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss_mod(y, y_pred) 
        errors.append(err)
        if min_err - err > tol:  
            min_err = err
        else:  
            print(
                f'Stop descent! iteration: {n_iter}, weights: {W}, logloss: {min_err}')
            stop_chek = False
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if verbose:
            if n_iter % view_ind == 0:
                print(n_iter, W, err)
    return W, min_err, n_iter

In [52]:
W = eval_model(X_st, y, alpha=0.6, tol=0.00001, verbose=True)

1000 [-8.650831   -1.25372329 -1.54901597  7.41955336] 0.28239658432599735
2000 [-12.40065236  -1.54519184  -2.73360596  10.40076601] 0.24130322780777833
3000 [-15.36100912  -1.79394114  -3.63840883  12.7942367 ] 0.215608798138124
4000 [-17.89496779  -2.01191968  -4.40078502  14.84932619] 0.19678626622267767
5000 [-20.12312962  -2.20577442  -5.0641727   16.65729995] 0.1822476349304427
6000 [-22.11355698  -2.38010899  -5.65216163  18.27197463] 0.17065959157189314
Stop descent! iteration: 6204, weights: [-22.49301224  -2.41343975  -5.76381384  18.57970581], logloss: 0.1685880929323425


In [53]:
def get_best_params(X,y,args):
    best_params=[]
    for arg in args:
        W,err,n_iter = eval_model(X, y, alpha=arg)
        best_params.append((arg,err,n_iter))
    best_params.sort(key=lambda x:x[1])
    print(f'best - alpha: {best_params[0][0]},\nresults:\nerr: {best_params[0][1]},\nn_iter: {best_params[0][2]}')
    return best_params[0]

In [54]:
alphas=[1, 0.5,0.1,0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005, 0.00001, 0.000005, 0.000001, 0.0000005, 0.0000001]
bp=get_best_params(X_st,y,alphas)
bp

Stop descent! iteration: 4, weights: [-0.01512682 -1.44851808  0.75644797  1.06048112], logloss: 0.9360085439942413
Stop descent! iteration: 6391, weights: [-20.79292735  -2.26433611  -5.26248927  17.20072759], logloss: 0.17820649249087703
Stop descent! iteration: 5853, weights: [-8.53503588 -1.24557698 -1.51126618  7.32961021], logloss: 0.2838539721501477
Stop descent! iteration: 6493, weights: [-6.18756485 -1.10137381 -0.7192765   5.56154533], logloss: 0.3198397227902269
Stop descent! iteration: 6823, weights: [-2.01966427 -0.97319445  0.84001927  2.78391711], logloss: 0.43337113257078075
Stop descent! iteration: 4134, weights: [-0.6636217  -0.82018794  1.11647506  1.7462896 ], logloss: 0.4980620278791868
Stop descent! iteration: 3017, weights: [ 0.06153887 -0.68603638  0.85254687  1.2130508 ], logloss: 0.5558287097554773
Stop descent! iteration: 3104, weights: [ 0.19306961 -0.69531514  0.74058514  1.21943805], logloss: 0.5765738129341405
Stop descent! iteration: 6890, weights: [ 0.3

(0.5, 0.17820649249087703, 6391)

Лучший результат получаем при a = 0.5

In [55]:
alphas=np.arange(1,10)/10
bp=get_best_params(X_st,y,alphas)
bp

Stop descent! iteration: 5853, weights: [-8.53503588 -1.24557698 -1.51126618  7.32961021], logloss: 0.2838539721501477
Stop descent! iteration: 6187, weights: [-12.59825223  -1.5614873   -2.79460722  10.55999424], logloss: 0.23942165779557922
Stop descent! iteration: 6534, weights: [-16.06978213  -1.85457135  -3.85261322  13.36879382], logloss: 0.21006752362747183
Stop descent! iteration: 6535, weights: [-18.71799582  -2.08333293  -4.64651497  15.51714508], logloss: 0.19120600628315837
Stop descent! iteration: 6391, weights: [-20.79292735  -2.26433611  -5.26248927  17.20072759], logloss: 0.17820649249087703
Stop descent! iteration: 6204, weights: [-22.49301224  -2.41343975  -5.76381384  18.57970581], logloss: 0.1685880929323425
Stop descent! iteration: 6, weights: [ 9.91945996e-04 -8.78773470e-01  8.74063808e-01  1.19348750e+00], logloss: 0.5868163760298956
Stop descent! iteration: 4, weights: [ 0.06140228 -1.1126246   0.76192585  1.11769668], logloss: 0.6865501501467717
Stop descent! 

(0.6, 0.1685880929323425, 6204)

Лучший результат получаем при a = 0.6

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred_proba.

In [56]:
def calc_pred_proba(w, x): 
    pred_proba = sigmoid(np.dot(w, x))
    return pred_proba

In [57]:
W,_err,_it = eval_model(X_st, y, alpha=0.6, verbose=True)

1000 [-8.650831   -1.25372329 -1.54901597  7.41955336] 0.28239658432599735
2000 [-12.40065236  -1.54519184  -2.73360596  10.40076601] 0.24130322780777833
3000 [-15.36100912  -1.79394114  -3.63840883  12.7942367 ] 0.215608798138124
4000 [-17.89496779  -2.01191968  -4.40078502  14.84932619] 0.19678626622267767
5000 [-20.12312962  -2.20577442  -5.0641727   16.65729995] 0.1822476349304427
6000 [-22.11355698  -2.38010899  -5.65216163  18.27197463] 0.17065959157189314
Stop descent! iteration: 6204, weights: [-22.49301224  -2.41343975  -5.76381384  18.57970581], logloss: 0.1685880929323425


4. Создайте функцию calc_pred, возвращающую предсказанный класс. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred.

In [58]:
def calc_pred(w,x, prob_lim=0.5):    
    pred_proba = sigmoid(np.dot(w, x))
    pred=np.zeros_like(pred_proba)
    for idx, prob in enumerate(pred_proba):
        if prob>prob_lim:
            pred[idx]=1
    return pred

In [59]:
y_pred=calc_pred(W, X_st)
y_pred

array([0., 0., 1., 0., 1., 0., 1., 0., 0., 1.])

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [60]:
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score

In [61]:
def my_accuracy(real, pred):
    all_res=len(real)
    trues=0
    for i in range(all_res):
        trues+=int(real[i]==pred[i])
    return trues/all_res

In [62]:
my_accuracy(y,y_pred)

0.9

In [63]:
def my_confusion_matrix(real, pred):
    n_classes=len(np.unique(real))
    all_res=len(real)
    conf_matr=np.zeros((n_classes,n_classes), dtype='int')
    for i in range(all_res):
        conf_matr[int(real[i])][int(pred[i])] += 1
    return conf_matr  

In [64]:
my_confusion_matrix(y,y_pred)

array([[5, 0],
       [1, 4]])

In [65]:
def my_precision(real, pred):
    tn, fp, fn, tp = my_confusion_matrix(real, pred).ravel()
    return tp/(tp+fp)

In [66]:
my_precision(y,y_pred)

1.0

In [67]:
def my_recall(real, pred):
    tn, fp, fn, tp = my_confusion_matrix(real, pred).ravel()
    return tp/(tp+fn)

In [68]:
my_recall(y,y_pred)

0.8

In [69]:
def my_f1(real, pred, beta=1):
    return (1+beta**2)*my_precision(real, pred)*my_recall(y,y_pred)/(beta**2*my_precision(real, pred)+my_recall(y,y_pred))

In [70]:
my_f1(y,y_pred)

0.888888888888889

6. Могла ли модель переобучиться? Почему?

Модель логистической регрессии может переобучиться, в связи с тем что используется сигмоида для преобразования в вероятность предсказания линейной модели. Так как сигмоида не имеет максимума и минимума, а только асимптоты, градиентный спуск не может достичь оптимального решения с помощью градиентных шагов доводя веса до все более экстремальных значений, пытаясь достичь нулевых потерь (вероятность этого события увеличивается с увеличением размерности данных). 
В качестве способа регуляризации можно использовать раннюю остановку работы модели, а так же L1 и L2 регуляризацию.